# Visualizing flight paths to Chicago

starting from here : https://makersportal.com/blog/2018/8/23/airport-destination-mapper-in-real-time-with-python

airport data from here : https://ourairports.com/help/data-dictionary.html 

I wonder if I should grab all the flight info that I can, and just highlight the chicago ones?  That might look cooler.  I could probably also remove the map if I get enough.

In [1]:
import numpy as np
import pandas as pd

import requests
import re 

from dateutil.parser import parse
from datetime import *


In [2]:
#file from ourairports.com
airports_df = pd.read_csv('airports.csv')
airports_df

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.949200,-151.695999,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR
5,322127,00AS,small_airport,Fulton Airport,34.942803,-97.818019,1100.0,NaN,US,US-OK,Alex,no,00AS,NaN,00AS,NaN,NaN,NaN
6,6527,00AZ,small_airport,Cordes Airport,34.305599,-112.165001,3810.0,NaN,US,US-AZ,Cordes,no,00AZ,NaN,00AZ,NaN,NaN,NaN
7,6528,00CA,small_airport,Goldstone /Gts/ Airport,35.350498,-116.888000,3038.0,NaN,US,US-CA,Barstow,no,00CA,NaN,00CA,NaN,NaN,NaN
8,324424,00CL,small_airport,Williams Ag Airport,39.427188,-121.763427,87.0,NaN,US,US-CA,Biggs,no,00CL,NaN,00CL,NaN,NaN,NaN
9,322658,00CN,heliport,Kitchen Creek Helibase Heliport,32.727374,-116.459742,3350.0,NaN,US,US-CA,Pine Valley,no,00CN,NaN,00CN,NaN,NaN,NaN


In [46]:
def convertTime(tlist, today):
    out = []
    for x in tlist :
        val = x[0].replace('\xa0',' ')
        try:
            dt = parse(val, default = today)
            out.append(dt.hour + dt.minute/60. + dt.second/3600.)
        except:
            out.append(val)
            #out.append(np.nan)
    return out

def getIdent(alist):
    out = []
    outIdent = []
    lat = []
    lon = []
    for x in alist:
        try:
            p1 = x[0].rfind('(')
            p2 = x[0].rfind(')')
            val = x[0][p1+1:p2]
            v1 = val
            v2 = val
            p3 = val.find('/')
            if (p3 != -1):
                v1 = val[:p3]
                v2 = val[p3+2:]
            #out.append(val)
            out.append(x[0])
            row = airports_df.loc[(airports_df['ident'] == v1) | 
                                 (airports_df['gps_code'] == v1) |
                                 (airports_df['iata_code'] == v1) | 
                                 (airports_df['local_code'] == v1) |
                                 (airports_df['ident'] == v2) | 
                                 (airports_df['gps_code'] == v2) |
                                 (airports_df['iata_code'] == v2) | 
                                 (airports_df['local_code'] == v2) ]
            if (len(row) >=1):
                lat.append(row['latitude_deg'].values[0])
                lon.append(row['longitude_deg'].values[0])
                outIdent.append(row['ident'].values[0])
            else:
                lat.append(-999.)
                lon.append(-999.)
                outIdent.append('foo')

        except:
            out.append('foo')
            outIdent.append('foo')
            lat.append(-999)
            lon.append(-999)
    return out, outIdent, lon, lat
              

In [47]:
key_match_start = '<table class="prettyTable fullWidth"'
key_match_end = '</table>'
def getAirport(airportCode, maxOffset, today):
    offset = 0
    origin = []
    originIdent = []
    lat = []
    lon = []
    arrive = []
    depart = []
    while offset < maxOffset:
        #for requests to Flight Aware
        page = requests.get('https://flightaware.com/live/airport/'+airportCode+'/arrivals?;offset='+str(offset)+';order=actualarrivaltime;sort=DES')
        p1 = page.text.find(key_match_start)
        p2 = page.text.rfind(key_match_end)
        flight_html = page.text[p1:p2]
        if (len(flight_html) > 0):
            flight_df = pd.read_html(flight_html)[0]
            o, oi, lo, la = getIdent(flight_df.iloc[:, flight_df.columns.get_level_values(1)=='Origin'].values)
            if ('Sorry. No matching flights found' in o[0]):
                break
            d = convertTime(flight_df.iloc[:, flight_df.columns.get_level_values(1)=='Departure'].values, today)
            a = convertTime(flight_df.iloc[:, flight_df.columns.get_level_values(1)=='Arrival'].values, today)

            origin.extend(o)
            originIdent.extend(oi)
            lon.extend(lo)
            lat.extend(la)
            depart.extend(d)
            arrive.extend(a)

            offset += 20
        else:
            break

        #print(offset)
    return origin, originIdent, lon, lat, depart, arrive

In [ ]:
nrows = 10000
istart = airports_df.loc[airports_df['ident'] == '2XA1'].index
today = datetime.now()
print(today)

for index, row in airports_df.iterrows():
    if (index >= istart):
        airport = row['ident']
        origin, originIdent, lon, lat, depart, arrive = getAirport(airport, nrows, today)
        print(index, airport, len(origin), len(originIdent), len(lon), len(lat), len(arrive), len(depart))
        if (len(origin) > 0):
            out_df = pd.DataFrame()
            out_df['departure_airport_name'] = origin
            out_df['departure_airport'] = originIdent
            out_df['departure_time'] = depart
            out_df['departure_longitude'] = lon
            out_df['departure_latitude'] = lat
            out_df['arrival_airport'] = np.full_like(origin, airport)
            out_df['arrival_time'] = arrive
            out_df['arrival_longitude'] = np.full_like(lon, row['longitude_deg'])
            out_df['arrival_latitude'] = np.full_like(lat, row['latitude_deg'])
            out_df.to_csv('flightData/'+airport+'_flights.csv', index=False)

2020-02-08 20:36:25.581270
3623 2XA1 0 0 0 0 0 0
3624 2XA2 0 0 0 0 0 0
3625 2XA3 0 0 0 0 0 0
3626 2XA4 0 0 0 0 0 0
3627 2XA5 0 0 0 0 0 0
3628 2XA6 0 0 0 0 0 0
3629 2XA7 0 0 0 0 0 0
3630 2XA8 0 0 0 0 0 0
3631 2XA9 0 0 0 0 0 0
3632 2XS0 0 0 0 0 0 0
3633 2XS1 0 0 0 0 0 0
3634 2XS2 0 0 0 0 0 0
3635 2XS3 0 0 0 0 0 0
3636 2XS4 0 0 0 0 0 0
3637 2XS5 0 0 0 0 0 0
3638 2XS6 0 0 0 0 0 0
3639 2XS7 0 0 0 0 0 0
3640 2XS8 0 0 0 0 0 0
3641 2XS9 0 0 0 0 0 0
3642 2Y0 0 0 0 0 0 0
3643 2Y1 0 0 0 0 0 0
3644 2Y2 0 0 0 0 0 0
3645 2Y3 0 0 0 0 0 0
3646 2Z1 0 0 0 0 0 0
3647 2Z2 0 0 0 0 0 0
3648 2Z3 0 0 0 0 0 0
3649 2Z5 0 0 0 0 0 0
3650 2Z6 0 0 0 0 0 0
3651 30AK 0 0 0 0 0 0
3652 30AL 0 0 0 0 0 0
3653 30AR 0 0 0 0 0 0
3654 30AZ 0 0 0 0 0 0
3655 30CA 0 0 0 0 0 0
3656 30CL 0 0 0 0 0 0
3657 30CO 0 0 0 0 0 0
3658 30F 0 0 0 0 0 0
3659 30FA 0 0 0 0 0 0
3660 30FD 0 0 0 0 0 0
3661 30FL 0 0 0 0 0 0
3662 30GA 0 0 0 0 0 0
3663 30ID 0 0 0 0 0 0
3664 30II 0 0 0 0 0 0
3665 30IL 0 0 0 0 0 0
3666 30IN 0 0 0 0 0 0


### Plot these with plotFlights.py